In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import FunctionTransformer, KBinsDiscretizer, StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler, OneHotEncoder

Importation des modules et rechargement du module helpers :

In [2]:
import importlib
import helpers

importlib.reload(helpers)
from helpers import categorize_imc

Chargement des données depuis un fichier CSV :

In [3]:
data_path = "Dataset_Brief.csv"
df = pd.read_csv(data_path)

Calcul du taux de valeurs manquantes et suppression des lignes avec des valeurs manquantes :

In [4]:
missing_rate = df.isna().sum()/df.shape[0]
missing_rate
df = df.dropna()

Identification et suppression des doublons :

In [5]:
duplicates = df[df.duplicated(keep=False)]
print(duplicates)
index_to_remove = 581
df = df.drop(index_to_remove)
df = df.reset_index(drop=True)
print(df.shape)

     age   sex    bmi  children smoker     region    charges
195   19  male  30.59         0     no  northwest  1639.5631
581   19  male  30.59         0     no  northwest  1639.5631
(1337, 7)


Définition des catégories pour la catégorisation de l'IMC (Indice de Masse Corporelle) :

In [6]:
categories = {
    'Underweight': (0, 18.5),
    'Normal Weight': (18.5, 24.9),
    'Overweight': (25, 29.9),
    'Obesity Class I': (30, 34.9),
    'Obesity Class II': (35, 39.9),
    'Obesity Class III': (40, float('inf'))
}

Catégorisation de l'IMC et suppression de la colonne 'bmi' :

In [7]:
df['imc_category'] = df['bmi'].apply(categorize_imc)
df = df.drop('bmi', axis=1)
df

NameError: name 'categories' is not defined

Sélection des caractéristiques (features) :

Séparation des données en ensembles d'entraînement et de test :

In [ ]:
X = df.drop('charges', axis=1)
y = df.charges
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

Identification des colonnes numériques et catégorielles :

In [ ]:
num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

Création des pipelines de prétraitement :

In [ ]:
onehotscale_pipeline = make_pipeline(OneHotEncoder(), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

Prétraitement des données avec des pipelines :

In [ ]:
preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

Définition de la grille de recherche d'hyperparamètres :

In [ ]:
param_grid = {'lasso__alpha': list(range(36, 40))}

Construction du pipeline final :

In [ ]:
polyscale_pipeline = make_pipeline(PolynomialFeatures(2))
my_final_pipeline = make_pipeline(preprocessing, polyscale_pipeline)

Entraînement du pipeline final sur les données d'entraînement :

In [ ]:
my_final_pipeline.fit(X_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder()),
                                                                  ('robustscaler',
                                                                   RobustScaler(with_centering=False))]),
                                                  ['sex', 'smoker', 'region',
                                                   'imc_category']),
                                                 ('numerical',
                                                  Pipeline(steps=[('robustscaler',
                                                                   RobustScaler(with_centering=False))]),
                                                  ['age', 'children'])])),
                ('pipeline',
                 Pipeline(steps=[('polynomialfeatures',
                                  PolynomialFeatures())]))])

In [ ]:
feature_names = my_final_pipeline.get_feature_names_out(X.columns)
feature_names

array(['1', 'categorical__sex_female', 'categorical__sex_male',
       'categorical__smoker_no', 'categorical__smoker_yes',
       'categorical__region_northeast', 'categorical__region_northwest',
       'categorical__region_southeast', 'categorical__region_southwest',
       'categorical__imc_category_Normal Weight',
       'categorical__imc_category_Obesity Class I',
       'categorical__imc_category_Obesity Class II',
       'categorical__imc_category_Obesity Class III',
       'categorical__imc_category_Overweight',
       'categorical__imc_category_Underweight',
       'categorical__imc_category_None', 'numerical__age',
       'numerical__children', 'categorical__sex_female^2',
       'categorical__sex_female categorical__sex_male',
       'categorical__sex_female categorical__smoker_no',
       'categorical__sex_female categorical__smoker_yes',
       'categorical__sex_female categorical__region_northeast',
       'categorical__sex_female categorical__region_northwest',
       'c

Création du modèle de régression Lasso avec GridSearchCV :

In [ ]:
model = make_pipeline(
    my_final_pipeline,
    Lasso(random_state=42, max_iter=100000)
)

Entraînement du modèle Lasso avec recherche de grille :

In [ ]:
model = GridSearchCV(model, param_grid,cv =5)
model.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('columntransformer',
                                                         ColumnTransformer(transformers=[('categorical',
                                                                                          Pipeline(steps=[('onehotencoder',
                                                                                                           OneHotEncoder()),
                                                                                                          ('robustscaler',
                                                                                                           RobustScaler(with_centering=False))]),
                                                                                          ['sex',
                                                                                           'smoker',
                                                                                           'region',
                                                                                           'imc_category']),
                                                                                         ('numerical',
                                                                                          Pipeline(steps=[('robustscaler',
                                                                                                           RobustScaler(with_centering=False))]),
                                                                                          ['age',
                                                                                           'children'])])),
                                                        ('pipeline',
                                                         Pipeline(steps=[('polynomialfeatures',
                                                                          PolynomialFeatures())]))])),
                                       ('lasso',
                                        Lasso(max_iter=100000,
                                              random_state=42))]),
             param_grid={'lasso__alpha': [36, 37, 38, 39]})

Évaluation du modèle sur les données d'entraînement :

In [ ]:
grid_score = model.score(X_train, y_train)
best_alpha = model.best_params_['lasso__alpha']

Prédiction sur les données de test :

In [ ]:
y_pred = model.predict(X_test)

Évaluation du modèle :

In [ ]:
r2_cleaned = r2_score(y_test, y_pred)
mse_cleaned = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle après suppression des valeurs extrêmes:")
print(grid_score)
print(model.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")

Performance du modèle après suppression des valeurs extrêmes:
0.8559809626895564
{'lasso__alpha': 36}
R2 Score: 0.9231
MAE: 2051.44
RMSE: 3338.52


Sauvegarde du modèle :

In [ ]:
with open('modele.pkl', 'wb') as file:
    pickle.dump(grid_search, file)